In [5]:
from html import unescape
from bs4 import BeautifulSoup
import re
import requests
import html
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
names = {'norma_part1.shtml': 'Норма', 'ochered.shtml': 'Очередь', 
         'marina_part1.shtml': 'Тридацатая любовь Марины', 'roman_part0.shtml': 'Роман',
        'serdtsa4_part01.shtml': 'Сердца четырех', 'salo_part1.shtml': 'Голубое сало',
        'led_part01.shtml': 'Лед', 'bro_part01.shtml': 'Путь Бро', 
        '23000_part08.html': '23000 главы из романа', 'telluria.html': 'Теллурия глава из романа'}

Для начала напишем функцию, которая будет очищать извлечённый текст от тэгов.

In [4]:
def clean_text(text):
    regex_tag = re.compile('<.*?>', re.DOTALL)
    regex_tag_t = re.compile('{.*?}', re.DOTALL)
    regex_space = re.compile('\s{2,}', re.DOTALL)
    clean_t = regex_space.sub('', text)
    clean_t = regex_tag_t.sub ('', clean_t)
    clean_t = regex_tag.sub('', clean_t)
    clean_t = html.unescape(clean_t)
    return clean_t

Теперь напишем функцию, которая со страницы "Романы" на сайте Сорокина будет получать ссылки на "стартовые страницы" романов и возвращать список ссылок

In [3]:
def get_names(genre):
    req = session.get('https://srkn.ru/' + genre, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    novel_links = soup.find_all('span', {'class': 'field-content'})
    links_start = []
    for link in novel_links:
        new = re.findall('href="texts/(.*).?html', str(link))
        new = str(new[0]) + 'html'
        links_start.append(new)
    return links_start

Теперь напишем функцию, которая берёт список ссылок и проходится со стартовой ссылки до конца романа через page-next. Каждая страница добавляется в один общий список texts.

In [32]:
def get_texts(links_start):
    texts = []
    for num, l in enumerate(links_start):
        pages = ''
        nex = l
        while len(nex)!=0:
            req = session.get('https://srkn.ru/texts/' + nex, headers={'User-Agent': ua.random})
            page = req.text
            soup = BeautifulSoup(page, 'html.parser')
            text_pro = soup.find_all('div', {'class': 'field-item even'})
            #Сайт Сорокина устроен так, что либо текст идёт четвертым в div class field-item even, либо последним
            #Последним, элементов меньше 4.
            if len(text_pro) > 3:
                new_text = clean_text(str(text_pro[3]))
            else:
                new_text = clean_text(str(text_pro[-1]))
            pages = pages + new_text
            links = soup.find_all('a', {'class': 'page-next'})
            nex = re.findall('href="/texts/(.*).?html', str(links))
            if len(nex) != 0:
                nex = str(nex[0]) + 'html'
            
        title = names[l]
        to_tuple = (title, pages)
        texts.append(to_tuple)
        
    return texts

Ну и наконец запись в файл и вызов функций.

In [33]:
links_romany = get_names('romany')
texts_romany = get_texts(links_romany)


In [40]:
type(texts_romany)

list

In [17]:
with open('sorokin_corpus.txt', 'w') as f:
    for text in texts:
        f.write(text)
    f.close()

И сразу на основе текстов сделаем модель markovify и сохраним её.

In [37]:
import json

In [38]:
a = json.dumps(texts_romany)

In [39]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(texts_romany, f, ensure_ascii = False)